In [17]:
import os
import ee

ee.Initialize()

#====================parameters
root = 'E:/itc-stars/code/'
imagepath = 'baseClassifiedImagesPath.txt'


# ======================================load the classified   images
os.chdir(root)
ID_Image = open(root + imagepath, 'r')
lstimages = ID_Image.readlines()
lstimages = list(map(str.strip,lstimages))

# ==============================geometry and region to clip and export results
image = ee.Image(lstimages[0])
geom = image.geometry()
region = image.geometry().bounds().getInfo()['coordinates'][0]
#print(geom.getInfo())


# =====================================initialize 5 images to stack the classification
IC1 = ee.Image()
IC2 = ee.Image()
IC3 = ee.Image()
IC4 = ee.Image()
IC5 = ee.Image()


In [22]:
#concatenar las imagenes como bandas
for fname in lstimages:
    image2 = ee.Image(fname)
   
    if fname.find('class1') > 0:
        IC1 = IC1.addBands(image2)
    elif fname.find('class2') > 0:
        IC2 = IC2.addBands(image2)
    elif fname.find('class3') >  0:
        IC3 = IC3.addBands(image2)
    elif fname.find('class4') > 0:
        IC4 = IC4.addBands(image2)
    elif fname.find('class5') > 0:
        IC5 = IC5.addBands(image2)

In [23]:
print(IC4.getInfo())

{'bands': [{'id': 'constant', 'data_type': {'max': 0, 'precision': 'int', 'min': 0, 'type': 'PixelType'}, 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0], 'crs': 'EPSG:4326'}, {'id': 'constant_1', 'data_type': {'precision': 'double', 'type': 'PixelType'}, 'crs_transform': [4.49157642059761e-06, 0.0, -5.2363965721196255, 0.0, -4.49157642059761e-06, 12.217415749104195], 'dimensions': [20801, 20458], 'crs': 'EPSG:4326'}, {'id': 'constant_2', 'data_type': {'precision': 'double', 'type': 'PixelType'}, 'crs_transform': [4.49157642059761e-06, 0.0, -5.2363965721196255, 0.0, -4.49157642059761e-06, 12.217415749104195], 'dimensions': [20801, 20458], 'crs': 'EPSG:4326'}, {'id': 'constant_3', 'data_type': {'precision': 'double', 'type': 'PixelType'}, 'crs_transform': [4.49157642059761e-06, 0.0, -5.2363965721196255, 0.0, -4.49157642059761e-06, 12.217415749104195], 'dimensions': [20801, 20458], 'crs': 'EPSG:4326'}, {'id': 'constant_4', 'data_type': {'precision': 'double', 'type': 'PixelType'}, 'crs_t

In [24]:
#=================================== Accumulate the weights
# initialize images
IC1sum = ee.Image(0)
IC2sum = ee.Image(0)
IC3sum = ee.Image(0)
IC4sum = ee.Image(0)
IC5sum = ee.Image(0)

# Accumulate the weiths -- hacer un reducer que sume

IC1sum = IC1.reduce(ee.Reducer.sum())
IC2sum = IC2.reduce(ee.Reducer.sum())
IC3sum = IC3.reduce(ee.Reducer.sum())
IC4sum = IC4.reduce(ee.Reducer.sum())
IC5sum = IC5.reduce(ee.Reducer.sum())


In [25]:
print(IC1sum.getInfo())

{'bands': [{'id': 'sum', 'data_type': {'precision': 'double', 'type': 'PixelType'}, 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0], 'crs': 'EPSG:4326'}], 'type': 'Image'}


In [26]:
IC = IC1sum.addBands(IC2sum).addBands(IC3sum).addBands(IC4sum).addBands(IC5sum)
print("*************************************************\n",IC.getInfo())

*************************************************
 {'bands': [{'id': 'sum', 'data_type': {'precision': 'double', 'type': 'PixelType'}, 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0], 'crs': 'EPSG:4326'}, {'id': 'sum_1', 'data_type': {'precision': 'double', 'type': 'PixelType'}, 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0], 'crs': 'EPSG:4326'}, {'id': 'sum_2', 'data_type': {'precision': 'double', 'type': 'PixelType'}, 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0], 'crs': 'EPSG:4326'}, {'id': 'sum_3', 'data_type': {'precision': 'double', 'type': 'PixelType'}, 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0], 'crs': 'EPSG:4326'}, {'id': 'sum_4', 'data_type': {'precision': 'double', 'type': 'PixelType'}, 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0], 'crs': 'EPSG:4326'}], 'type': 'Image'}


In [27]:
# ================================Extract the maximum  # majority voting

cond11 = IC.expression('b("sum") >= b("sum_1") && b("sum") >= b("sum_2")')
cond12 = IC.expression('b("sum") >= b("sum_3") && b("sum") >= b("sum_4")')
cond1 = cond11.And(cond12)
cond21 = IC.expression('b("sum_1") > b("sum") && b("sum_1") >= b("sum_2")')
cond22 = IC.expression('b("sum_1") >= b("sum_3") && b("sum_1") >= b("sum_4")')
cond2 = cond21.And(cond22)
cond31 = IC.expression('b("sum_2") > b("sum") && b("sum_2") > b("sum_1")')
cond32 = IC.expression('b("sum_2") >= b("sum_3") && b("sum_2") >= b("sum_4")')
cond3 = cond31.And(cond32)

cond41 = IC.expression('b("sum_3") > b("sum") && b("sum_3") > b("sum_1")')
cond42 = IC.expression('b("sum_3") > b("sum_2") && b("sum_3") >= b("sum_4")')
cond4 = cond41.And(cond42)   # And in uppercase

cond51 = IC.expression('b("sum_4") > b("sum") && b("sum_4") > b("sum_1")')
cond52 = IC.expression('b("sum_4") > b("sum_3") && b("sum_4") > b("sum_2")')
cond5 = cond51.And(cond52)

In [43]:
geom = image.geometry()
output = ee.Image(0)
output = output.clip(geom)

#1 - Maize	400	225  #2 - Millet	400	225  #3 - Peanut	400	225  #4 - Sorghum	400	225 #5- Cotton	520	299

output = output.where(cond1, 1) 
output = output.where(cond2, 2) 
output = output.where(cond3, 3) 
output = output.where(cond4, 4) 
output = output.where(cond5, 5) 
output = output.clip(geom)
print("*************************************************\n",output.getInfo())

*************************************************
 {'bands': [{'id': 'constant', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0], 'origin': [-6, 11], 'data_type': {'max': 5, 'precision': 'int', 'min': 0, 'type': 'PixelType'}, 'dimensions': [2, 2], 'crs': 'EPSG:4326'}], 'properties': {'system:footprint': {'coordinates': [[[-5.236394875491211, 12.125524882429849], [-5.2363943178275125, 12.12552480600459], [-5.189681947295634, 12.12552482139066], [-5.142969554934702, 12.125524809925233], [-5.142967407474762, 12.125526675183306], [-5.142965100943807, 12.12552819565501], [-5.142964941872711, 12.125528980980178], [-5.142964941872711, 12.21741346750349], [-5.142966771190352, 12.217415608568812], [-5.142968361901392, 12.217417841080646], [-5.142969157256931, 12.21741800028432], [-5.189681947295634, 12.217418043281285], [-5.2363943178275125, 12.217418017065055], [-5.236396468816098, 12.217416149409656], [-5.236398779137263, 12.217414626711694], [-5.23639893846976, 12.217413840741836], [-5.23639

In [47]:
from IPython.display import Image  # To display image thumbnails.
mpalette = ['000000','bdc3c7','1e8449','d68910','2ecc71','f9e79f']
url = output.getThumbUrl({'min':1, 'max':5})
print(url)
Image(url=url)

EEException: Invalid color value.

In [ ]:
# load our image 
image = ee.Image("users/rosamaguilar/tutorial/subset")
# display the image, The maximun pixel value is 2048 because it has 11-bit per pixel.
Image(url=image.getThumbUrl({'min':0, 'max':2048}))